In [125]:
from pprint import pprint

In [89]:
from train_retriever_classification import *

encoder_question = BertEncoder(bert_question, max_question_len_global)
encoder_paragarph = BertEncoder(bert_paragraph, max_paragraph_len_global)
ret = Retriver(encoder_question, encoder_paragarph, tokenizer)

checkpoint_callback = ModelCheckpoint(
    filepath='out/{epoch}-{val_loss:.2f}-{val_acc:.2f}',
    save_top_k=10,
    verbose=True,
    monitor='val_acc',
    mode='max'
)

early_stopping = EarlyStopping('val_acc', mode='max')

trainer = pl.Trainer(
    gpus=8,
    distributed_backend='dp',
    val_check_interval=0.000001,
    min_epochs=1,
    checkpoint_callback=checkpoint_callback,
    early_stop_callback=early_stopping,
    gradient_clip_val=0.5
)
ret_trainee = RetriverTrainer(ret)

tmp = torch.load('out/crossentropy-epoch=1-val_loss=0.70-val_acc=0.62.ckpt')

ret_trainee.load_state_dict(tmp['state_dict'])

<All keys matched successfully>

In [137]:
q= 'Do you love me?'
s1= 'I love you.'
s2= 'You hate me.'
s3= 'It aint much but it is honesdt work.'

In [138]:
ret_trainee.retriever.predict(q, [s1, s2, s3])

(['I love you.', 'You hate me.', 'It aint much but it is honesdt work.'],
 array([0.37463212, 0.2758383 , 0.27113175], dtype=float32),
 array([0, 1, 2]))

In [93]:
from elasticsearch import Elasticsearch

In [94]:
es = Elasticsearch(
        [{"host": "es-covidfaq.dev.dialoguecorp.com", "port": 443}],
        use_ssl=True,
        verify_certs=True,
    )
if not es.ping():
    raise ValueError(
        "Connection failed, please start server at localhost:9200 (default)"
    )

In [95]:
def search_section_index(es, index, query, topk):
    res = es.search(
        {
            "query": {
                "multi_match": {"query": query, "fields": ["section", "content"],}
            },
            "size": topk,
        },
        index=index,
    )
    return res

In [141]:
secindex = "en-covid-section-index"
topk_sec = 3

In [142]:
qs = [
#     'What is Covid-19?',
#     'What are the symptoms of Covid 19? ',
#     'How does Covid-19 spread?',
#     'When should I go to the hospital?',
#     'How many cases in Montreal?',
    'What should I do if I have fever?',
#     'What is the incubation period for COVID-19?',
#     'How can I protect myself from the covid-19?',
#     'How can I make the difference between a cold and covid19?',
    'Where can I get tested?',
#     'Is it true that warm kills Coronavirus?',
]

In [146]:
rs_rerank

(['As often as possible, but especially: • Before I touch my face (eyes, mouth, nose) • After I cough, sneeze or blow my nose • Before and after I provide care to a close friend or family member • When my hands are visibly dirty or after I touch a dirty object • Before and after I prepare meals • Before and after I eat • After using the toilet • Before and after going to a public place\n\n\n\n\n',
  'Adults at risk of severe respiratory symptoms: • I have a fever higher than 38 °C/100.4 °F. • I am in an at-risk category for severe respiratory complications (senior or person with chronic cardiac and/or pulmonary problems, immune deficiency or diabetes).\n\n\n\n\n',
  'Stay at home and avoid contact with others Wash your hands frequently Follow good respiratory hygiene practices: –– Cover your mouth and nose when coughing or sneezing (elbow, upper arm or disposable paper tissue) Call the 1-877-644-4545 coronavirus hotline to report your symptoms and follow the instructions you are given 

In [170]:
ret_trainee.retriever.bert_question_encoder.bert = 

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [144]:
for q in qs:

    rs = search_section_index(es, secindex, q, topk_sec)["hits"]["hits"]
    print('question: ', q, '\n')
    txts = [' '.join(i['_source']['content']) + '\n'*5 for i in rs]
    print('Elastic Search: ')
    pprint(txts[:3])
    
    rs_rerank = ret_trainee.retriever.predict(q, txts)
    print('=' * 50, '\n')
    print('BERT: ')
    pprint(rs_rerank[0][:3], rs_rerank[0][:3])
    
    print('-' * 100)

question:  What should I do if I have fever? 

Elastic Search: 
['Adults at risk of severe respiratory symptoms: • I have a fever higher than '
 '38 °C/100.4 °F. • I am in an at-risk category for severe respiratory '
 'complications (senior or person with chronic cardiac and/or pulmonary '
 'problems, immune deficiency or diabetes).\n'
 '\n'
 '\n'
 '\n'
 '\n',
 'As often as possible, but especially: • Before I touch my face (eyes, mouth, '
 'nose) • After I cough, sneeze or blow my nose • Before and after I provide '
 'care to a close friend or family member • When my hands are visibly dirty or '
 'after I touch a dirty object • Before and after I prepare meals • Before and '
 'after I eat • After using the toilet • Before and after going to a public '
 'place\n'
 '\n'
 '\n'
 '\n'
 '\n',
 'Stay at home and avoid contact with others Wash your hands frequently Follow '
 'good respiratory hygiene practices: –– Cover your mouth and nose when '
 'coughing or sneezing (elbow, upper arm or di

AttributeError: 'list' object has no attribute 'write'

In [154]:
import json

In [165]:
hits = []
c = 0
with open('natq/natq_clean.json', 'r') as f:
    for l in f:
        d = json.loads(l)
        if d['dataset'] == 'dev':
            q = d['question']
            txts = d['right_paragraphs'] + d['wrong_paragraphs'][:2]
            c += 1
            if ret_trainee.retriever.predict(q, txts)[0][0] == txts[0]:
                hits.append(1)
            else:
                hits.append(0)
        if c > 100:
            break

KeyboardInterrupt: 

In [166]:
sum(hits) / len(hits)

0.35294117647058826

In [167]:
len(hits)

34

In [157]:
q = d['question']
txts = d['right_paragraphs'] + d['wrong_paragraphs'][:2]

In [161]:
txts = [txts[2], txts[1], txts[0]]

In [162]:
q

'when did computer become widespread in homes and schools'

In [163]:
txts

['<P> In developed nations , computers achieved semi-ubiquity during the 1980s as they made their way into schools , homes , business , and industry . Automated teller machines , industrial robots , CGI in film and television , electronic music , bulletin board systems , and video games all fueled what became the zeitgeist of the 1980s . Millions of people purchased home computers , making household names of early personal computer manufacturers such as Apple , Commodore , and Tandy . To this day the Commodore 64 is often cited as the best selling computer of all time , having sold 17 million units ( by some accounts ) between 1982 and 1994 . </P>',
 "<P> Over 39 percent of all tornado - related deaths and many injuries come from residents of mobile homes . It is a widespread myth that tornadoes are `` attracted '' to mobile home parks , and cause the most fatalities there because they hit there the most . This is not true . However , mobile homes do often have poor construction and do

In [164]:
ret_trainee.retriever.predict(q, txts)

(['<P> Home computers were a class of microcomputers entering the market in 1977 , and becoming common during the 1980s . They were marketed to consumers as affordable and accessible computers that , for the first time , were intended for the use of a single nontechnical user . These computers were a distinct market segment that typically cost much less than business , scientific or engineering - oriented computers of the time such as the IBM PC , and were generally less powerful in terms of memory and expandability . However , a home computer often had better graphics and sound than contemporary business computers . Their most common uses were playing video games , but they were also regularly used for word processing , doing homework , and programming . </P>',
  '<P> In developed nations , computers achieved semi-ubiquity during the 1980s as they made their way into schools , homes , business , and industry . Automated teller machines , industrial robots , CGI in film and television 